In [6]:
import pandas as pd
import numpy as np
%matplotlib inline

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv('ts_hr_feb_2020.csv.zip', parse_dates=['date'])

data['hour'] = data['date'].dt.hour



last_sunday = pd.to_datetime("2020-02-23")
train = data[data['date'] < last_sunday]
val = data[data['date'] >= last_sunday]

In [12]:
def prep_seqs(df, l=24, h=24):
    X = []
    Y = []
    for i in range(l,df.shape[0]-h):
        f = df.iloc[i-l:i][['sales_value', 'hour']] #hoje
        y = df.iloc[i:i+h]['sales_value'] # 1 dia depois

        X.append(f.values)
        Y.append(y.values)

    X = np.array(X)
    Y = np.array(Y)

    X = np.swapaxes(X, 1,2)
    
    return torch.from_numpy(X).float(), torch.from_numpy(Y).float()

In [14]:
X, Y = prep_seqs(train)
Xt, Yt = prep_seqs(val)

mean_train = X.mean(0, keepdims=True)
mean_train[:, 1, :] = 0
std_train = X.std(0,  unbiased=False, keepdims=True)
std_train[:, 1, :] = 1

meanY_train = Y.mean(0, keepdims=True)
stdY_train = Y.std(0, unbiased=False, keepdims=True)

X -= mean_train
X /= std_train

Xt -= mean_train
Xt /= std_train


Y -= meanY_train
Y /= stdY_train


X = X.cuda()
Y = Y.cuda()


In [18]:
class CNN(nn.Module):
    def __init__(self, conv_filters, kernel_size, embedding_size, max_pool_kernel_size):
        super(CNN, self).__init__()
        self.kernel_size = kernel_size
        self.embedding_size = embedding_size
        self.conv_filters = conv_filters
        self.max_pool_kernel_size = max_pool_kernel_size
     
        self.hour_emb = nn.Embedding(24, self.embedding_size)
        self.conv = nn.Conv1d(self.embedding_size+1, self.conv_filters, self.kernel_size)
        self.maxpool = nn.MaxPool1d(self.max_pool_kernel_size)
        self.flat = nn.Flatten()
        
        len_conv_out = (24-self.kernel_size+1)
        #print(len_conv_out)
        len_maxpool_out = int(np.floor((len_conv_out - self.max_pool_kernel_size) / self.max_pool_kernel_size) + 1)
        #print(len_maxpool_out)
        self.out = nn.Linear(self.conv_filters*len_maxpool_out, 24)
        
    def forward(self, input):
        hour_emb = self.hour_emb(input[:,-1,:].long().cuda())
        hour_emb = torch.transpose(hour_emb, 1, 2)
        nums = input[:,:-1,:].cuda()
        
        
        inputs = torch.cat([nums, hour_emb], dim=1)
        
        print("Conv1d input", inputs.shape)
        output = self.conv(inputs)
        print("Conv1d output", output.shape)
        output = self.maxpool(output)
        print("MaxPool1d output", output.shape)
        output = self.flat(output)
        print("Flatten output", output.shape)
        output = self.out(output)
        print("CNN output", output.shape)
        return output.squeeze()

In [19]:
a = CNN(10,2,3,2).cuda()
a(X).shape, Y.shape

Conv1d input torch.Size([479, 4, 24])
Conv1d output torch.Size([479, 10, 23])
MaxPool1d output torch.Size([479, 10, 11])
Flatten output torch.Size([479, 110])
CNN output torch.Size([479, 24])


(torch.Size([479, 24]), torch.Size([479, 24]))

In [41]:
%%time


def run_cnn(params):
    conv_filters, kernel_size, embedding_size, max_pool_kernel_size, lr = params
    #print(params)
    conv_filters = int(conv_filters)
    kernel_size = int(kernel_size)
    embedding_size = int(embedding_size)
    max_pool_kernel_size = min(kernel_size, int(max_pool_kernel_size))

    
    
    total_p = np.zeros((Yt.shape[0], Yt.shape[1], 10))

    
    
    for seed in range(10):
        torch.manual_seed(seed)
        np.random.seed(seed)

        cnn_ = CNN(conv_filters, kernel_size, embedding_size, max_pool_kernel_size).cuda() # LEMBRE DO .cuda() !!!!!!
        criterion = nn.L1Loss()
        cnn_optimizer = optim.Adam(cnn_.parameters(), lr=lr)


        for i in range(100):
            cnn_.zero_grad()

            o = cnn_(X)
            loss = criterion(o, Y)
            loss.backward()
            cnn_optimizer.step()


            p = cnn_(Xt)
            p = p.detach().cpu()

            p *= stdY_train
            p += meanY_train
            p = p.numpy().squeeze()


        total_p[:, :, seed] = p
        

    e = mean_absolute_error(Yt.numpy(), total_p.mean(axis=-1))

    print("\conv_filters = {}, kernel_size = {}, embedding_size = {}, max_pool_kernel_size = {}, lr = {}".format(conv_filters, kernel_size, embedding_size, max_pool_kernel_size, lr))
    print("Ensemble Avg Score = {}".format(e))
    
    return e


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


In [42]:
from skopt import gp_minimize

space = [(10, 100),
         (2,16),
         (1, 200),
         (2,4),
         (1e-4, 1e-1, 'log-uniform')]

res = gp_minimize(run_cnn, space, random_state=3, verbose=1, n_calls=50)

Iteration No: 1 started. Evaluating function at random point.
\conv_filters = 16, kernel_size = 14, embedding_size = 25, max_pool_kernel_size = 3, lr = 0.002047320462003159
Ensemble Avg Score = 330.49238783255754
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 4.8826
Function value obtained: 330.4924
Current minimum: 330.4924
Iteration No: 2 started. Evaluating function at random point.
\conv_filters = 12, kernel_size = 3, embedding_size = 50, max_pool_kernel_size = 2, lr = 0.012147053888882771
Ensemble Avg Score = 337.837858237905
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 3.8597
Function value obtained: 337.8379
Current minimum: 330.4924
Iteration No: 3 started. Evaluating function at random point.
\conv_filters = 23, kernel_size = 8, embedding_size = 44, max_pool_kernel_size = 3, lr = 0.0030054380132902175
Ensemble Avg Score = 333.88660189731064
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 4.2763
Function value obta

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 9.1701
Function value obtained: 325.1448
Current minimum: 324.8381

conv_filters = 10, kernel_size = 2, embedding_size = 60, max_pool_kernel_size = 2, lr = 0.0020513056084200515
Ensemble Avg Score = 324.8381097431025
